In [11]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import copy
import math
import time
from multiprocessing import Pool
from joblib import Parallel, delayed
import multiprocessing
import datetime
from time import sleep
# from PIL import Image
# from PIL import ImageTk
import random
import glob
from tqdm import tqdm
import tkinter as tk
from tkinter import filedialog as fd
import sys

In [12]:
def foccal(pt, step, minar):
    stepar=[]
    opar=[]
    varwin=4.5*step/((2*math.pi)**0.5)
    for i in range(1,12):
        #print(i)
        x=minar+(i-1)*step
        if np.round(x)==1:
            x==1.00
        #print(x)
        stepar.append(np.round(x,2))
        func=(1/(((2*math.pi)**0.5)*varwin))*math.exp(-(x-pt)**2/(2*(varwin**2)))
        func=np.round(func,8)
        opar.append(func)
    foclist=opar
    gpar=[]
    steplist=stepar
    brack=[]
    focar=np.round(np.max(foclist)-foclist)
    for i in range(0,10):
        gpar.append(2*int(focar[i])+1)
    return(gpar)

def newmaskimg(image,array, blurparam):
    
    step=np.round((np.max(array)-np.min(array))/10,3)
    minar=np.min(array)
    maxar=np.max(array)
    
    stepar=[]
    for i in range(1,12):
        #print(i)
        x=minar+(i-1)*step
#         if np.round(x)==1:
#             x==1.00
            #print(x)
#         stepar.append(np.round(x,2))
        stepar.append(x)
#     print(stepar)
    
    brack=[]
    for i in range(0,10):
        brack.append([stepar[i],stepar[i+1]])
    brack[9][1] = 1
#     print(brack)
    
    masks=[]
    temp_table = np.zeros((img.shape[0],img.shape[1]))
    for i in range(0,10):
        newarr=copy.copy(array)
        newarr[np.where(newarr>brack[i][1])]=0
        newarr[np.where(newarr<brack[i][0])]=0
        newarr[np.where(newarr>0)]=1
        #mask=np.zeros(image.shape)
        #  print(mask[:,:,0].shape)
        resized = cv.resize(newarr, (img.shape[1],img.shape[0]), interpolation = cv.INTER_AREA)
#         print(resized.shape)
#         temp_table += i*resized[:,:,0]
        temp_table += i*resized
        mask=np.dstack([resized,resized,resized])
        #print('mask',mask.shape)
        masks.append(mask)
        
    imar=[]
    for i in range(4):
        imar.append(cv.GaussianBlur(image,(blurparam*i+1,blurparam*i+1),0))
        #print(10*i+1)
        
#     print(temp_table)
    return(imar,masks, temp_table)

def renderopfast(blimg,blarray,image, masks):
    
    oparray=[]
    for j in range(10):
        recons=np.zeros(image.shape)
        for i in range(10):
            if blarray[j][i]==1:
                recons=recons+masks[i]*blimg[0]
            elif blarray[j][i]==3:
                recons=recons+masks[i]*blimg[1]
            elif blarray[j][i]==5:
                recons=recons+masks[i]*blimg[2]
            elif blarray[j][i]==7:
                recons=recons+masks[i]*blimg[3]
        
        oparray.append(recons)
        
    return(np.uint8(oparray))

def read_inputs():
    #Directory 
    imarr = []    
    npyarr = []

    #Reading Image Files
    for filename in glob.glob(dirr + '/' + '*.jpg'):
        imtemp = cv.imread(filename)
        imarr.append(imtemp)

    #Reading Numpy Arrays
    for filename in glob.glob(dirr + '/' + '*.npy'):
        arrtemp = np.load(filename)
        npyarr.append(arrtemp)
        
    return imarr, npyarr

# Mouse Tracking
# posx = 0
# posy = 0
def mouse_move(event,x,y,flags, param):
    global posx, posy
    if event == cv.EVENT_MOUSEMOVE:
        posx = x
        posy = y


# Display Function
def output_win():
    global posx, posy
    posx = 0
    posy = 0
    
    outputarray = opimarr
    luptable = opluptable
    
    cv.namedWindow('output')
    
    cv.setMouseCallback('output',mouse_move)
    
    # Moving across width is x value
    # Moving along height is y value
    # table[frame_index][row or y][col or x]
    
    frame = outputarray[0][9]
    i = 0
    blur_area = 9
    comp_var = 9
    
    font = cv.FONT_HERSHEY_SIMPLEX 
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
    
#     print(len(imarr))
        
    while(i < len(imarr)):

        frame = outputarray[i][blur_area]
        cv.imshow('output',frame)        
        time.sleep(0.04)

        i += 1

        if i == len(imarr):
            i = 0

        comp_var = np.uint8(luptable[i][posy][posx])

        if comp_var < 10:
            blur_area = comp_var

        if cv.waitKey(20) & 0xFF == 27:
            break


    cv.destroyAllWindows()
    
    return

def genpreview(blurfac):
    
    global img, arr1
    img = cv.imread(dirr + '/00000.jpg')
    arr1 = np.load(dirr + '/00000.npy')
    
    global imarr
    imarr, npyarr = read_inputs()
    
    minar=np.min(npyarr[0])
    step=np.round((np.max(npyarr[0])-np.min(npyarr[0]))/10,3)

    blarrs = []
    for i in range(10):
        blarrs.append(foccal(minar+i*step, step, minar))
        
    global opims, lup_tab

    blurimages,masks, lup_tab=newmaskimg(imarr[0], npyarr[0], blurfac)
    opims=renderopfast(blurimages,blarrs,imarr[0], masks)
    return

def preview_win():
    blurfac = int(blurVariable.get())
#     print(blurVariable.get())
    genpreview(blurfac)
    
    global posx, posy
    posx = 0
    posy = 0
    
    cv.namedWindow('Preview output')
    #cv.resizeWindow('output', 600,600)
    
    cv.setMouseCallback('Preview output',mouse_move)
    
    # Moving across width is x value
    # Moving along height is y value
    # table[frame_index][row or y][col or x]
    i=0
    blur_area = 9
    comp_var = 9
    font = cv.FONT_HERSHEY_SIMPLEX 
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
    
    while(i == 0):
            
        frame = opims[blur_area]
        cv.imshow('Preview output',frame)        
#         time.sleep(0.04)
            
        comp_var = np.uint8(lup_tab[posy][posx])

        if comp_var < 10:
            blur_area = comp_var

        if cv.waitKey(20) & 0xFF == 27:
            break


    cv.destroyAllWindows()
    
    return

def Application():
    global img, arr1
    img = cv.imread(dirr + '/00000.jpg')
    arr1 = np.load(dirr + '/00000.npy')  
    print(sys.getsizeof(img))
    print(sys.getsizeof(arr1))
    
    global imarr
#     start_time = time.process_time()
    imarr, npyarr = read_inputs()
#     print("--- %s seconds ---" % (time.process_time() - start_time))
    print(sys.getsizeof(imarr))
    print(sys.getsizeof(npyarr))

    minar=np.min(npyarr[0])
    step=np.round((np.max(npyarr[0])-np.min(npyarr[0]))/10,3)

    blarrs = []
    for i in range(10):
        blarrs.append(foccal(minar+i*step, step, minar))
        
    global opimarr, opluptable
    opimarr = []
    opluptable = []
    for index in tqdm(range(len(imarr)), unit="Frames", desc="Processing"):

        minar=np.min(npyarr[index])
        maxar=np.max(npyarr[index])
        step=np.round((np.max(npyarr[index])-np.min(npyarr[index]))/10,3)

        blurimages,masks, lup_tab=newmaskimg(imarr[index], npyarr[index], 6)

        #Per Frame image output
        opims=renderopfast(blurimages,blarrs,imarr[index], masks)

        #Append opims onto the output image array
        opimarr.append(opims)

        #Append luptable
        opluptable.append(lup_tab)
        
    return
        

  

In [13]:
  
def browse_directory():
    # Allow user to select a directory and store it in global var
    # called dirr
    global dirr
    dirr = fd.askdirectory()
    lbl1.set(dirr)

def view():
    # Initializing the display window
    view_win = tk.Toplevel()
    view_win.title("Video Parameters")
    
    # Button to Pre-process and Run application
    runApp_but = tk.Button(view_win, text = "Process Video", width = 10, command = Application)
    runApp_but.grid(row=1, column=0)
    
    # Button to display output
    output_but = tk.Button(view_win, text = "Output Video", width = 10, command = output_win)
    output_but.grid(row=2, column=0)
    
    # Button to close and destroy current window
    close_but = tk.Button(view_win, text = "Close", width = 10, command = view_win.destroy)
    close_but.grid(row=5, column=0)
    
    quitApp_but = tk.Button(view_win, text = "Quit App", width = 10, command = window.destroy)
    quitApp_but.grid(row=6, column=0)
        

def preview():
    # Initializing the display window
    pv_win = tk.Toplevel()
    pv_win.title("Preview Parameters")
    
    input1 = tk.Label(pv_win, text = "Choose the blur level")
    input1.grid(row=0, column=0)
    
    global blurVariable
    blurVariable = tk.StringVar(pv_win)
    blurVariable.set(6) # default value

    blurOption = tk.OptionMenu(pv_win, blurVariable, 2, 6, 10, 20, 30, 40, 50, 100)
    blurOption.grid(row=0, column=1)
        
    # Button to display preview
    generate_but = tk.Button(pv_win, text = "Generate", width = 10, command = preview_win)
    generate_but.grid(row=2, column=0)
    
    # Button to close and destroy current window
    close_but = tk.Button(pv_win, text = "Close", width = 10, command = pv_win.destroy)
    close_but.grid(row=5, column=0)
    
    quitApp_but = tk.Button(pv_win, text = "Quit App", width = 10, command = window.destroy)
    quitApp_but.grid(row=6, column=0)
    
    

In [14]:
#Set up GUI
window = tk.Tk()  #Makes main window
window.wm_title("Focus_GUI: Root Window")
window.config(background="#FFFFFF")

global lbl1
lbl1 = tk.StringVar()
lbl1.set("Choose the required Directory")

# Button to choose Video directory with the .jpg and .npy files
but_direc = tk.Button(window, text = "Directory", width = 10, command = browse_directory)
but_direc.grid(row=0, column=1)

# Label to dynamically display the current directory    
label1 = tk.Label(window, textvariable = lbl1)
label1.grid(row=0, column=3)
    
but_preview = tk.Button(window, text = "Preview", width = 10, command=preview)
but_preview.grid(row=2, column=1)

but_view = tk.Button(window, text = "View Video", width = 10, command=view)
but_view.grid(row=4, column=1)

but_quitApp = tk.Button(window, text = "Quit", width = 10, command = window.destroy)
but_quitApp.grid(row=6, column=1)

In [15]:
window.mainloop()  #Starts GUI

Processing:   0%|          | 0/71 [00:00<?, ?Frames/s]

442496
589936
640
640


Processing: 100%|██████████| 71/71 [00:24<00:00,  2.90Frames/s]


In [162]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name, x):
    info('function f')
    print('hello', name)
    global a
    a=x*x
    
def f2(name, x):
    info('function f2')
    print('hello', name)
    global b
    b=x*x
    

if __name__ == '__main__':
    global a, b
    a=0
    b=0

#     info('main line')
    p = Process(target=f, args=('bob',2,))
    p2 = Process(target=f2, args=('bob',3,))
    p.start()
    p2.start()
    p.join()
    p2.join()
    
    print(a, b)

0 0
